In [1]:
import os, sys
os.sys.path.append('..')
from recommender.utils import mongodbutils as mdb
from recommender.utils.ItemMetadata import ExplicitDataFromCSV
from recommender.utils.eval_utils import compute_auc, compute_ap
from recommender.recommender.recommenderMF import RecommenderMF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

D:\python\Python35\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\python\Python35\lib\site-packages\numpy\.libs\libopenblas.BNVRK7633HSX7YVO2TADGR4A5KEKXJAW.gfortran-win_amd64.dll
D:\python\Python35\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
D:\python\Python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ratings_csv = 'D:/PycharmProjects/recommender/data/ml-20m/ratings_sanitized.csv'
user_map_csv = 'D:/PycharmProjects/recommender/data/ml-20m/user_map.csv'
item_map_csv = 'D:/PycharmProjects/recommender/data/ml-20m/item_map.csv'
md_csv = 'D:/PycharmProjects/recommender/data/ml-20m/metadata.csv'
stats_csv = 'D:/PycharmProjects/recommender/data/ml-20m/stats.csv'
exp_data = ExplicitDataFromCSV(True, ratings_csv=ratings_csv, user_map_csv=user_map_csv, item_map_csv=item_map_csv, md_csv=md_csv, stats_csv=stats_csv)

In [4]:
# ratings_csv = 'D:/PycharmProjects/recommender/data/ml-20m/ratings.csv'
# movies_csv = 'D:/PycharmProjects/recommender/data/ml-20m/movies.csv'
# d = ExplicitDataFromCSV(False,
#                         ratings_csv=ratings_csv, r_item_col='movieId', r_user_col='userId',
#                         r_rating_col='rating',
#                         metadata_csv=movies_csv, m_item_col='movieId')
# d.save('D:\\PycharmProjects\\recommender\\data\\ml-20m')

In [3]:
model_path = 'D:/PycharmProjects/recommender/recommender/models/model/07201914185949/'
rmf = RecommenderMF(mode='predict', model_path=model_path)

INFO:tensorflow:Restoring parameters from D:/PycharmProjects/recommender/recommender/models/model/07201914185949/variables\variables


In [5]:
F = rmf.predict(np.zeros(shape=(exp_data.M)),np.arange(exp_data.M))

In [6]:
D = exp_data.stats.merge(exp_data.md_df, left_index=True, right_on='item_cat', how='inner')
D = D.merge(pd.DataFrame({'features': [f for f in F['q']]}), left_index=True, right_index=True)

In [7]:
D.head()

,user,title,genres,features
item_cat,,,,
0,49695,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[-0.2656288726679784, -0.17109544103827748, -0..."
1,22243,Jumanji (1995),Adventure|Children|Fantasy,"[-0.3061244954752809, -0.11782419674165967, -0..."
2,12735,Grumpier Old Men (1995),Comedy|Romance,"[-0.13756294066794034, -0.13669127569012585, -..."
3,2756,Waiting to Exhale (1995),Comedy|Drama|Romance,"[-0.36046933471456105, -0.22796148700900676, -..."
4,12161,Father of the Bride Part II (1995),Comedy,"[-0.28201282531225036, -0.13627450819181525, -..."


In [8]:
Dviz = D[D.user>500]

In [9]:
from sklearn.manifold import TSNE
tsne = TSNE(n_iter=15000, metric="cosine")
Q = np.array([f for f in Dviz['features']])
Q_emb = tsne.fit_transform(Q)
Q_emb.shape

(4483, 2)

In [10]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [11]:
trace = go.Scatter(x=Q_emb[:,0], y=Q_emb[:,1], text=Dviz['title'], mode='markers')

In [50]:
Dviz.set_index(np.arange(len(Dviz)), inplace=True)
query_ids = np.array([f for f in Dviz.loc[Dviz.title.str.startswith('Star Wars')].index])
#query_ids = np.array([f for f in Dviz.loc[Dviz.genres.map(lambda p: 'Horror' in p.split('|'))].index])
query_emb = Q_emb[query_ids]
query_titles = Dviz.loc[query_ids]['title']
trace2 = go.Scatter(x=query_emb[:,0], y=query_emb[:,1], text=query_titles, mode='markers')
print(Q_emb)

[[ 39.464447    3.1750126]
 [-22.754042   77.21931  ]
 [ -7.9982085  50.95934  ]
 ...
 [ 51.133232   -3.636324 ]
 [ 17.521177   33.219578 ]
 [ -3.8764884  28.736738 ]]


In [51]:
fig = go.Figure(data=[trace, trace2])
py.offline.iplot(fig)